In [38]:
from ngram_finder import NGramFinder
import csv
import nltk
import glob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

## Change filepath and comment column index below

In [123]:
# this is the file for which we are interested in the ngrams
main_file = "../data/2020-11-20 Stadteingang Elbbrücken_Kommentare.csv"
csvfile = glob.glob(f"../data/*.csv")

ngrams = []
count_of_all_words = 0

for file in csvfile:
    # exclude file that we want to analyze
    # otherwise if something comes up very often in this case,
    # it might still seem popular even if it's only popular here
    if file == main_file:
        continue
    if 'Beiträge'in file:
        comment_index = 10
    if 'Kommentare' in file:
        comment_index = 3

    print("now looking at" + str(file))
    curr_ngrams, count = Gen_Freq_Ngrams(file, comment_index)
    ngrams.append(curr_ngrams)
    count_of_all_words += count
    
# generate absolute amount of ngrams coming up
ngramdict = {}
for file in ngrams:
    for ngram in file:
        current = ngramdict.get(ngram[0])
        if current == None:
            ngramdict[ngram[0]] = ngram[1]
        else:
            ngramdict[ngram[0]] += ngram[1]
# print(ngramdict)

# generate relative amount of ngrams
for key in ngramdict.keys():
    ngramdict[key] = ngramdict[key] / count_of_all_words

# sort keys, this allows to have a look at what are the most popular ngrams
sorted_keys = sorted(ngramdict, key=ngramdict.get)
sorted_dict = {}
for w in sorted_keys:
    sorted_dict[w] = ngramdict[w]
# print(sorted_dict)

# look at just the maximum value
max_value_ngram = max(sorted_dict, key = sorted_dict.get)

print("done with calculating base values")
main_ngrams, count = Gen_Freq_Ngrams(main_file, 3)
main_ngrams_dict = {}
for ngram in main_ngrams:
    main_ngrams_dict[ngram[0]] = ngram[1] / count
print("created dict for main case")
for key in main_ngrams_dict.keys():
    overallRating = ngramdict.get(key)
    #print(f"key: {key}, overall rating {overallRating}, main rating {main_ngrams_dict[key]}")
    if overallRating == None:
        # this ngram didn't show up in any of the other cases, so it is likely relevant
        if main_ngrams_dict[key] > 0.00025:
            print(f"Unique:  {key}")
    elif overallRating * 0.1 < main_ngrams_dict[key]:
        # this ngram was more popular here than in other cases, so it is likely relevant
        print(f"Higher Value: {key}")
    
#print(main_ngrams_dict)    
# filepath = '../data/2020-11-20 Stadteingang Elbbruecken_Kommentare.csv'



# for the elbbruecken file the comments are in the fourth column
#comment_index = 3

now looking at../data/2020-11-20 Jungfernstieg_Kommentare.csv
now looking at../data/2020-11-20 Jungfernstieg_Beiträge.csv
done with calculating base values
created dict for main case
Unique:  ('u.a.', 'gewerbliche')
Unique:  ('strom-', 'wasseranschlüssen?!')
Unique:  ('bootsanleger', 'etc.')
Unique:  ('zwecke', 'strom-')
Unique:  ('geben,', 'u.a.')
Unique:  ('gewerbliche', 'zwecke')
Unique:  ('etc.', 'geben,')
Unique:  ('billhorner', 'brückenstraße')
Unique:  ('brandshofer', 'deich')
Unique:  ('endlich', 'mal')
Unique:  ('ja', 'schon')
Unique:  ('neben', 'elbtower')
Unique:  ('city', 'nord')
Unique:  ('ja', 'eh')
Unique:  ('erhalten', 'bleiben')
Unique:  ('variante', 'a')
Unique:  ('a', 'variante')
Higher Value: ('immer', 'mehr')
Unique:  ('neue', 'moschee')
Unique:  ('entwurf', 'b')


In [113]:
def Gen_Freq_Ngrams(filepath, comment_index):

    ngf = NGramFinder()
    with open(filepath, 'r', encoding='utf8') as file:
        lines = list(csv.reader(file, delimiter=';'))

        comments = [line[comment_index] for line in lines]
        
        count_all_words = 0
        for comment in comments:
            count_all_words += len(comment.split(' '))
        count = ngf.get_ngram_count(comments, 2, [])
        # print("Höchste Häufigkeit eines N-Grams: " + str(max(count.values())))
        frequent_n_grams = list()
        # length_all_comments = 
        for text, count in count.items():
            if count >= 3:
                frequent_n_grams.append((text, count))
                #print(text)
    return frequent_n_grams, count_all_words           

In [ ]:
word_count_before = 5 #float('inf')
word_count_after = 3

for ngram in frequent_n_grams:
    print ("Suche nach " + str(ngram) + "\n")
    for context in ngf.get_contexts_for_ngram(comments, ngram, word_count_before, word_count_after):
        print("    " + context + "\n")
